In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import cv2
import numpy as np
from os.path import isfile

import cv2

def video2framesarray(videoinput):
    cap = cv2.VideoCapture(videoinput)
    frame_number = 0
    frame_array = []
    fps = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_array.append(frame)
        frame_number += 1

        if frame_number == 1:
            start_time = cv2.getTickCount()
        
    end_time = cv2.getTickCount()
    time = (end_time - start_time) / cv2.getTickFrequency()
    fps = frame_number / (time*10)

    cap.release()
    return frame_array, fps

In [2]:
def atmosdehaze(frame):
    # Convert the image to floating point representation
    hazy_image = frame.astype(np.float32) / 255.0

    # Estimate the atmospheric light
    dark_channel = np.min(hazy_image, axis=2)
    atmospheric_light = np.percentile(dark_channel, 99)

    # Estimate the transmission map
    transmission = 1 - 0.95 * dark_channel / (atmospheric_light + 1e-6)  # Add a small epsilon value

    # Clamp the transmission values to [0, 1]
    transmission = np.clip(transmission, 0, 1)

    # Estimate the scene radiance
    scene_radiance = np.zeros_like(hazy_image)
    for channel in range(3):
        scene_radiance[:, :, channel] = (hazy_image[:, :, channel] - atmospheric_light) / (transmission + 1e-6) + atmospheric_light  # Add a small epsilon value

    # Clamp the scene radiance values to [0, 1]
    scene_radiance = np.clip(scene_radiance, 0, 1)

    # Convert the scene radiance back to 8-bit representation
    scene_radiance = (scene_radiance * 255).astype(np.uint8)
    return scene_radiance

def dehaze_images(frame_array):
    dehazed_frames = []
    for frame in frame_array:
        dehazed_frame = atmosdehaze(frame)
        dehazed_frames.append(dehazed_frame)

    # Convert the dehazed frames to an array
    dehazed_array = np.array(dehazed_frames)
    return dehazed_array

In [4]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.202 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (AMD EPYC 7B12)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5016.7/8062.4 GB disk)


In [5]:
import cv2
import os
import shutil
from ultralytics import YOLO
def objectdectect(source_images):
    output_folder = 'temp_images'
    os.makedirs(output_folder, exist_ok=True)

    # Save the images from the array to the folder
    for i, image in enumerate(source_images):
        image_filename = os.path.join(output_folder, f'image_{i}.jpg')
        cv2.imwrite(image_filename, image)

    model = YOLO('yolov8n.pt')  # Pretrained YOLOv8n model
    model(source='temp_images', save=True, project='Project')
    #!yolo task=detect mode=predict model=yolov8n.pt source="temp_images"
    # Create an empty list to store the new images with bounding boxes
    output_images = []

    # Define the directory where the saved results are located
    project_directory = 'project'

    # List the files in the 'project/predict' directory
    result_files = os.listdir(os.path.join('/kaggle/working/Project/predict'))

    # Sort the image files by name
    sorted_result_files = sorted(result_files, key=lambda x: int(os.path.splitext(x)[0].split('_')[1]))


    # Load the sorted image files into an array
    output_images = [cv2.imread(os.path.join('/kaggle/working/Project/predict', filename)) for filename in sorted_result_files]
    shutil.rmtree('/kaggle/working/Project')
    return output_images

In [6]:
def dehazed2video(dehazed_array, pathOut, fps=40):
    size = (dehazed_array[0].shape[1], dehazed_array[0].shape[0])
    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(dehazed_array)):
        out.write(dehazed_array[i])
    out.release()

In [7]:
def main(input_video, output_video):
    frame_array,fps = video2framesarray(input_video)[0],video2framesarray(input_video)[1]
    dehazed_array = dehaze_images(frame_array)
    n = input("Do you want Object Detection: (y/n): ")
    if (n == 'y'):
        dehazed_array = objectdectect(dehazed_array)
        dehazed2video(dehazed_array, output_video,fps)
    else:
        dehazed2video(dehazed_array, output_video,fps)

if __name__ == '__main__':
    input_video = '/kaggle/input/hazetestvideo/hazevideo1.mp4'
    output_video = 'output_video.mp4' 
    main(input_video, output_video)

Do you want Object Detection: (y/n):  y


100%|██████████| 6.23M/6.23M [00:00<00:00, 61.7MB/s]

image 1/207 /kaggle/working/temp_images/image_0.jpg: 640x384 1 boat, 166.1ms
image 2/207 /kaggle/working/temp_images/image_1.jpg: 640x384 1 boat, 64.7ms
image 3/207 /kaggle/working/temp_images/image_10.jpg: 640x384 (no detections), 58.6ms
image 4/207 /kaggle/working/temp_images/image_100.jpg: 640x384 (no detections), 58.2ms
image 5/207 /kaggle/working/temp_images/image_101.jpg: 640x384 (no detections), 59.4ms
image 6/207 /kaggle/working/temp_images/image_102.jpg: 640x384 (no detections), 58.5ms
image 7/207 /kaggle/working/temp_images/image_103.jpg: 640x384 (no detections), 60.6ms
image 8/207 /kaggle/working/temp_images/image_104.jpg: 640x384 (no detections), 60.7ms
image 9/207 /kaggle/working/temp_images/image_105.jpg: 640x384 (no detections), 54.9ms
image 10/207 /kaggle/working/temp_images/image_106.jpg: 640x384 (no detections), 60.9ms
image 11/207 /kaggle/working/temp_images/image_107.jpg: 640x384 (no detections), 61.5ms
image 12/

In [ ]:
def video2framesarray(videoinput):
    cap = cv2.VideoCapture(videoinput)
    frame_number = 0
    frame_array = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_array.append(frame)
        frame_number += 1
    cap.release()
    return frame_array

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

In [ ]:
import cv2
import os
import shutil
from ultralytics import YOLO
def objectdectect(source_images):
    output_folder = 'temp_images'
    os.makedirs(output_folder, exist_ok=True)

    # Save the images from the array to the folder
    for i, image in enumerate(source_images):
        image_filename = os.path.join(output_folder, f'image_{i}.jpg')
        cv2.imwrite(image_filename, image)

    model = YOLO('yolov8n.pt')  # Pretrained YOLOv8n model
    model(source='temp_images', save=True, project='Project')
    #!yolo task=detect mode=predict model=yolov8n.pt source="temp_images"
    # Create an empty list to store the new images with bounding boxes
    output_images = []

    # Define the directory where the saved results are located
    project_directory = 'project'

    # List the files in the 'project/predict' directory
    result_files = os.listdir(os.path.join('/kaggle/working/Project/predict'))

    # Sort the image files by name
    sorted_result_files = sorted(result_files, key=lambda x: int(os.path.splitext(x)[0].split('_')[1]))


    # Load the sorted image files into an array
    output_images = [cv2.imread(os.path.join('/kaggle/working/Project/predict', filename)) for filename in sorted_result_files]
    shutil.rmtree('/kaggle/working/Project')
    return output_images

In [ ]:
vi = '/kaggle/input/hazetestvideo/hazevideo1.mp4'
source_images = video2framesarray(vi)
objectdectect(source_images)